In [1]:
%pip install pyautogui opencv-python opencv-python-headless
%pip install mss pillow pyscreeze pystray 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import mss
import time
import os
import threading
from PIL import Image, ImageDraw
import pystray

# Create a directory to save captured faces
if not os.path.exists('captured_faces'):
    os.makedirs('captured_faces')

def capture_screen():
    with mss.mss() as sct:
        monitor = sct.monitors[1]
        screenshot = sct.grab(monitor)
        img = np.array(screenshot)
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        return img_bgr

def detect_faces(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def save_faces(image, faces, frame_count):
    for i, (x, y, w, h) in enumerate(faces):
        face = image[y:y+h, x:x+w]
        face_filename = f'captured_faces/face_{frame_count}_{i}.png'
        cv2.imwrite(face_filename, face)

def main_loop():
    frame_count = 0
    last_saved_time = time.time()
    while not exit_event.is_set():
        start_time = time.time()
        current_time = time.time()

        screen_image = capture_screen()
        faces = detect_faces(screen_image)


        if current_time - last_saved_time >= 5:  # Save faces every 1 second
            save_faces(screen_image, faces, frame_count)
            last_saved_time = current_time

        # Draw rectangles around detected faces
        for (x, y, w, h) in faces:
            cv2.rectangle(screen_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Show the image with detected faces
        cv2.imshow('Detected Faces', screen_image)

        frame_count += 1

        end_time = time.time()
        elapsed_time = end_time - start_time
        delay = max(1, int((1/30 - elapsed_time) * 1000))  # Maintain ~30 FPS

        if cv2.waitKey(delay) & 0xFF == ord('q'):
            exit_event.set()
            break

    cv2.destroyAllWindows()

def create_image():
    # Generate an image for the system tray icon
    width = 64
    height = 64
    color1 = "white"
    color2 = "black"

    image = Image.new("RGB", (width, height), color1)
    dc = ImageDraw.Draw(image)
    dc.rectangle(
        [(width // 2, 0), (width, height // 2)],
        fill=color2)
    dc.rectangle(
        [(0, height // 2), (width // 2, height)],
        fill=color2)

    return image

def on_quit(icon, item):
    exit_event.set()
    icon.stop()

exit_event = threading.Event()

icon = pystray.Icon("face_detector")
icon.menu = pystray.Menu(pystray.MenuItem("Quit", on_quit))
icon.icon = create_image()
icon.title = "Face Detector"

# Start the face detection loop in a separate thread
detection_thread = threading.Thread(target=main_loop)
detection_thread.start()

# Run the system tray icon
icon.run()

# Wait for the detection thread to finish
detection_thread.join()


Exception in thread Thread-9 (main_loop):
Traceback (most recent call last):
  File "c:\Users\cholo\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\cholo\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\cholo\AppData\Local\Temp\ipykernel_15344\1443996309.py", line 42, in main_loop
  File "C:\Users\cholo\AppData\Local\Temp\ipykernel_15344\1443996309.py", line 24, in detect_faces
cv2.error: OpenCV(4.8.1) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xd8611878::Set<1,-1,-1>,struct cv::impl::A0xd8611878::Set<0,2,5>,3>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in 